In [50]:
import numpy as np
import random
import pandas as pd
import scipy.io
import matplotlib.pyplot as plt
from qiskit.quantum_info import DensityMatrix, random_density_matrix
from qiskit.quantum_info.operators import Operator
import tensorflow as tf

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score


# First Attempt, Bare Bones Neural Net

# Import the Dataset

In [54]:

QST_data = pd.read_csv("../data/qst_dataset.csv")
N = QST_data.shape[0]
QST_data.head()

,x0,x1,x2,x3,x4,x5,x6,x7,x8,x9,...,y22,y23,y24,y25,y26,y27,y28,y29,y30,y31
0,0.173828,0.274414,0.490234,0.061523,0.175781,0.259766,0.495117,0.069336,0.318359,0.133789,...,-0.011859,0.074632,0.014527,0.011859,-1.032135e-17,-0.156679,-0.083966,-0.074632,0.156679,2.956436e-18
1,0.269531,0.179688,0.277344,0.273438,0.293945,0.157227,0.257812,0.291016,0.179688,0.248047,...,-0.031100,0.056268,-0.136595,0.031100,-1.532738e-18,-0.145086,-0.101096,-0.056268,0.145086,1.936221e-18
2,0.204102,0.483398,0.210938,0.101562,0.256836,0.434570,0.201172,0.107422,0.196289,0.448242,...,-0.042311,-0.027411,0.095658,0.042311,2.765645e-18,-0.054666,0.046359,0.027411,0.054666,-2.587528e-18
3,0.389648,0.251953,0.078125,0.280273,0.411133,0.244141,0.084961,0.259766,0.488281,0.160156,...,0.104071,0.019399,0.073657,-0.104071,3.922884e-19,0.084765,-0.038264,-0.019399,-0.084765,-5.285627e-18
4,0.382812,0.276367,0.140625,0.200195,0.377930,0.229492,0.182617,0.209961,0.426758,0.207031,...,-0.165184,-0.025058,-0.023143,0.165184,5.761045e-18,-0.082392,0.058770,0.025058,0.082392,-3.947200e-18


In [55]:
# clip any small values to zero
eps = 1e-10

#create boolean mask
mask = QST_data.abs() < eps
QST_data[mask] =0

In [56]:
X_cols = []
y_cols = []

N_x = 36
N_y = 32
for i in range(N_x):
    X_cols.append(f"x{i}")
for i in range(N_y):
    y_cols.append(f"y{i}")

X = QST_data[X_cols]
y = QST_data[y_cols]

#split the data
X_train, X_test, y_train, y_test=  train_test_split(X, y, train_size=0.7, random_state=42)


In [57]:
X_train

,x0,x1,x2,x3,x4,x5,x6,x7,x8,x9,...,x26,x27,x28,x29,x30,x31,x32,x33,x34,x35
836,0.538086,0.216797,0.167969,0.077148,0.497070,0.247070,0.185547,0.070312,0.236328,0.476562,...,0.206055,0.236328,0.496094,0.090820,0.200195,0.212891,0.218750,0.350586,0.168945,0.261719
575,0.255859,0.140625,0.184570,0.418945,0.270508,0.142578,0.180664,0.406250,0.227539,0.177734,...,0.383789,0.339844,0.087891,0.190430,0.362305,0.359375,0.208008,0.091797,0.301758,0.398438
557,0.274414,0.290039,0.215820,0.219727,0.298828,0.320312,0.200195,0.180664,0.087891,0.506836,...,0.362305,0.442383,0.111328,0.056641,0.388672,0.443359,0.040039,0.141602,0.073242,0.745117
1235,0.328125,0.177734,0.158203,0.335938,0.301758,0.163086,0.184570,0.350586,0.317383,0.153320,...,0.285156,0.161133,0.273438,0.278320,0.266602,0.181641,0.403320,0.164062,0.368164,0.064453
1360,0.278320,0.300781,0.204102,0.216797,0.314453,0.280273,0.171875,0.233398,0.133789,0.475586,...,0.268555,0.177734,0.199219,0.348633,0.272461,0.179688,0.231445,0.345703,0.174805,0.248047
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1130,0.290039,0.147461,0.307617,0.254883,0.306641,0.153320,0.307617,0.232422,0.333008,0.146484,...,0.344727,0.094727,0.274414,0.255859,0.348633,0.121094,0.169922,0.392578,0.308594,0.128906
1294,0.151367,0.365234,0.085938,0.397461,0.113281,0.409180,0.090820,0.386719,0.339844,0.180664,...,0.058594,0.197266,0.143555,0.612305,0.062500,0.181641,0.483398,0.251953,0.112305,0.152344
860,0.206055,0.248047,0.131836,0.414062,0.206055,0.212891,0.144531,0.436523,0.286133,0.137695,...,0.191406,0.479492,0.127930,0.202148,0.215820,0.454102,0.142578,0.186523,0.325195,0.345703
1459,0.221680,0.118164,0.233398,0.426758,0.214844,0.112305,0.228516,0.444336,0.268555,0.047852,...,0.344727,0.215820,0.091797,0.320312,0.364258,0.223633,0.258789,0.169922,0.293945,0.277344


# Training the NN

In [58]:
QST_NN = tf.keras.Sequential([
    tf.keras.layers.InputLayer(shape=(N_x,)),
    tf.keras.layers.Dense(50, activation="relu"),       # 2 dense layers, TOBE modified
    tf.keras.layers.Dense(50, activation="relu"),       # 2 dense layers, TOBE modified
    tf.keras.layers.Dense(N_y)
])


In [59]:
QST_NN.compile(
    optimizer = 'adam',
    loss = 'mse',
    metrics=['mse']
)

In [60]:
QST_NN.fit(
    X_train, y_train,
    epochs=20,
    batch_size=32,
    validation_split=0.1
)

Epoch 1/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.0200 - mse: 0.0200 - val_loss: 0.0074 - val_mse: 0.0074
Epoch 2/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 818us/step - loss: 0.0070 - mse: 0.0070 - val_loss: 0.0060 - val_mse: 0.0060
Epoch 3/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 761us/step - loss: 0.0059 - mse: 0.0059 - val_loss: 0.0052 - val_mse: 0.0052
Epoch 4/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 793us/step - loss: 0.0051 - mse: 0.0051 - val_loss: 0.0045 - val_mse: 0.0045
Epoch 5/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 805us/step - loss: 0.0044 - mse: 0.0044 - val_loss: 0.0040 - val_mse: 0.0040
Epoch 6/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 806us/step - loss: 0.0040 - mse: 0.0040 - val_loss: 0.0037 - val_mse: 0.0037
Epoch 7/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 755us/step - loss: 0.0037 - mse: 0.0037 - val_loss: 0.0035 - val_mse: 0.0035
Epoch 8/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 802us/step - loss: 0.0035 - mse: 0.0035 - val_loss: 0.0034 - val_mse: 0.0034
Epoch 9/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 794us/step - loss

In [61]:
y_pred = QST_NN.predict(X_test)

test_loss, _ = QST_NN.evaluate(X_test, y_test)

print(f"test loss: {test_loss}")

19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 319us/step - loss: 0.0032 - mse: 0.0032
test loss: 0.003218843601644039


## Evaluation metrics

In [62]:
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse}")
print(f"R^2: : {r2}")


Mean Squared Error: 0.003218843504856142
R^2: : 0.43025660514831543


# Second Attempt: Using more Advanced Methods

## 1) Data PreProccessing
    
- Cholesky Decomposition on targets rho
- Principal component analysis on data X (frequencies)

## 2) Building Advanced Neural Network 
- Batch Normalisation
- Enforce positivity and Trace 1 on outputs
- Regularisation